In [236]:
# import python libraries
import pymysql
import pandas as pd
import numpy as np

In [237]:
parse_dates = ['date_added']
netflix_titles = pd.read_csv('mycsvfile.csv', parse_dates=parse_dates) # using pandas to read the csv file
netflix_titles['date_added'] = pd.to_datetime(netflix_titles['date_added'], errors='coerce') # change the date format to YYYY-MM-DD for column "date_added"
netflix_titles['date_added'] = netflix_titles['date_added'].dt.strftime("%Y-%m-%d")
fullList = netflix_titles['title'].values.tolist()
print(netflix_titles['date_added'].head())

0    2019-09-09
1    2016-09-09
2    2018-09-08
3    2018-09-08
4    2017-09-08
Name: date_added, dtype: object


In [238]:
showDim = netflix_titles.drop(['date_added', 'release_year', 'rating','listed_in'], axis=1) # delete the columns that are not needed
showDim = showDim.where((pd.notnull(showDim)), None) # convert empty values to "None" values
showDimList = []
for row in showDim.values.tolist():
    showDimList.append(tuple(row)) # format data into a list of tuples before inserting to database

# showDimList: show data for show_dim table

In [239]:
netflix_origionals = pd.read_csv('netflix_originals.csv') # using pandas to read the csv file
netflix_origionals = netflix_origionals[['Title','Seasons','Length','Netflix Exclusive Regions','Status']] # select the columns to keep
netflix_origionals = netflix_origionals.where((pd.notnull(netflix_origionals)), None) # convert empty values to "None" values
netflix_origionals.drop_duplicates(subset ="Title", 
                     keep = 'first', inplace = True) # delete duplicate values 
netflix_origionalsList = [tuple(l) for l in netflix_origionals.values.tolist()] # format data into a list of tuples before inserting to database
#print(netflix_origionalsList) #: data for original_dim table

In [240]:
netflix_stocks = pd.read_csv('NFLX.csv', parse_dates=['Date']) # using pandas to read the csv file

In [241]:
dateDim = pd.DataFrame({'date': pd.date_range(start='2002-05-23', end='2020-08-03')}) # create a dataframe that has dates ranging from 2002-05-23 to 2020-08-03
dateDim['date_id'] = dateDim.index + 1 # create date_id column and assign id numbers starting from 1
dateDim['date'] = dateDim['date'].dt.strftime("%Y-%m-%d") # format date column to YYYY-MM-DD
dateDim['year'] = pd.DatetimeIndex(dateDim['date']).year # using the year information from date colunn to create year column
dateDim = dateDim.reindex(columns=['date_id','date','year']) # re-arrange the order of columns
#dateDim.date = pd.to_datetime(dateDim.date)
dateDim.date_id = dateDim.date_id.astype(str) # convert data in date column to string

In [242]:
dateDimList = [tuple(l) for l in dateDim.values.tolist()] # format data into a list of tuples before inserting to database
# dateDimList: data for date_dim table

In [243]:
dateDim.date = pd.to_datetime(dateDim.date) # convert data in date column to datatype date

factsStockDF = pd.merge(netflix_stocks, dateDim, left_on='Date', right_on='date', how='inner') # inner join dataframes netflix_stocks and dataDim on date
factsStockDF = factsStockDF.drop(['Date', 'date', 'year'], axis=1) # delete columns that are not needed
factsStockDF = factsStockDF.reindex(columns=['date_id','Open','High','Low','Close','Adj Close', 'Volume']) # re-arrange the order of columns
# factsStockDF['Date'] = factsStockDF['Date'].dt.strftime("%Y-%m-%d")


In [244]:
factsStockList = [tuple(l) for l in factsStockDF.values.tolist()] # format data into a list of tuples before inserting to database
# factsStockList: data for facts_stock_prices table

In [245]:
factsRating = pd.merge(netflix_titles, netflix_origionals, left_on='title', right_on='Title', how='left') # left join dataframes netflix_titles and netflix_originals on titles
factsRating.date_added = pd.to_datetime(factsRating.date_added) # convert data in date_added column to datatype date

In [246]:
factsRatingDF = pd.merge(factsRating, dateDim, left_on='date_added', right_on='date', how='left') # left join dataframes factsRating and dateDim on date_added/date

In [247]:
factsRatingDF = factsRatingDF[['show_id','title','Title','date_id','rating']] # select only columns needed 
# factsRatingDF['date_id'] = factsRatingDF['date_id'].astype(str)

In [248]:
factsRatingDF = factsRatingDF.where((pd.notnull(factsRatingDF)), None) # convert empty values to "None" values

In [249]:
factsRatingList = [tuple(l) for l in factsRatingDF.values.tolist()] # format data into a list of tuples before inserting to database
#print(factsRatingList) # data for facts_IMDB_rating table

In [250]:
import pymysql

# Establish the database connection
db = pymysql.connect(
    host="localhost",
    user="root",
    password="",
    database="netflix_datawarehouse"
)

cursor = db.cursor()

# Drop the table if it already exists
cursor.execute("DROP TABLE IF EXISTS show_dim;")

# create table with attributes based on the dimensional model
createTable1 = """CREATE TABLE show_dim(
                 show_id INT not null,
                 type VARCHAR(255) null,
                 title VARCHAR(255) not null,
                 director VARCHAR(255) null,
                 cast MEDIUMTEXT null,
                 country VARCHAR(255) null,
                 duration VARCHAR(255) null,
                 description MEDIUMTEXT null,
                 PRIMARY KEY (show_id, title));"""

cursor.execute(createTable1)

insertData1 = "INSERT INTO show_dim VALUES (%s,%s,%s,%s,%s,%s,%s,%s);" # insert data into table
cursor.executemany(insertData1, showDimList)
db.commit()

db.close() # close the connection to mysql

In [251]:
db = pymysql.connect(
    host="localhost",
    user="root",
    password="",
    database="netflix_datawarehouse"
)
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS original_dim;") # delete the table if it already exists

# create table with attributes based on the dimensional model
createTable2 = """CREATE TABLE original_dim(
                  original_title VARCHAR(255) not null,
                  seasons VARCHAR(255) null,
                  length VARCHAR (255) null,
                  regions VARCHAR (255) null,
                  status VARCHAR (255) null,
                  PRIMARY KEY (original_title));"""

cursor.execute(createTable2)

insertData2 = "INSERT INTO original_dim VALUES (%s,%s,%s,%s,%s);" # insert data into table
cursor.executemany(insertData2, netflix_origionalsList)
db.commit()

db.close() # close the connection to mysql

In [252]:
# create date_dim table and insert data into it
db = pymysql.connect(
    host="localhost",
    user="root",
    password="",
    database="netflix_datawarehouse"
)
cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS date_dim;") # delete the table if it already exists

# create table with attributes based on the dimensional model
createTable3 = """CREATE TABLE date_dim(
                  date_id VARCHAR(50) not null,
                  date DATE null,
                  year INT null,
                  PRIMARY KEY (date_id));"""

cursor.execute(createTable3)

insertData3 = "INSERT INTO date_dim VALUES (%s,%s,%s);" # insert data into table
cursor.executemany(insertData3, dateDimList)
db.commit()

db.close() # close the connection to mysql

In [253]:
# create facts_imdb_rating table and insert data into it
db = pymysql.connect(
    host="localhost",
    user="root",
    password="",
    database="netflix_datawarehouse"
)
cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS facts_imdb_rating;")

# Tạo bảng mới
createTable5 = """
CREATE TABLE facts_imdb_rating (
    show_id INT NOT NULL,
    title VARCHAR(255) NOT NULL,
    original_title VARCHAR(255),
    date_id VARCHAR(50),
    rating FLOAT,
    PRIMARY KEY (show_id),
    FOREIGN KEY (show_id) REFERENCES show_dim(show_id),
    FOREIGN KEY (original_title) REFERENCES original_dim(original_title),
    FOREIGN KEY (date_id) REFERENCES date_dim(date_id)
);
"""
cursor.execute(createTable5)

insertData5 = "INSERT INTO facts_imdb_rating VALUES (%s,%s,%s,%s,%s);" # insert data into table
cursor.executemany(insertData5, factsRatingList)
db.commit()

db.close() # close the connection to mysql

In [254]:

# Kết nối với cơ sở dữ liệu
db = pymysql.connect(
    host="localhost",
    user="root",
    password="",
    database="netflix_datawarehouse"
)
cursor = db.cursor()

# Tạo bảng movie_genre

# Chuẩn bị dữ liệu cho movie_genre
# Ví dụ: netflix_titles có cột "show_id" và "listed_in"
netflix_titles['genres'] = netflix_titles['listed_in'].str.split(', ')

# Tạo danh sách các thể loại duy nhất
unique_genres = set(genre for genres in netflix_titles['genres'] for genre in genres)
genre_dict = {genre: idx+1 for idx, genre in enumerate(unique_genres)}

# Chèn các thể loại vào bảng genre_dim
cursor.execute("DROP TABLE IF EXISTS genre_dim;")
create_genre_dim = """
CREATE TABLE genre_dim (
    genre_id INT AUTO_INCREMENT,
    genre_name VARCHAR(255) NOT NULL,
    PRIMARY KEY (genre_id)
);
"""
cursor.execute(create_genre_dim)

insert_genre_dim = "INSERT INTO genre_dim (genre_name) VALUES (%s);"
cursor.executemany(insert_genre_dim, [(genre,) for genre in unique_genres])
db.commit()

cursor.execute("DROP TABLE IF EXISTS show_genre;")

create_movie_genre = """
CREATE TABLE show_genre (
    show_id INT NOT NULL,
    genre_id INT NOT NULL,
    PRIMARY KEY (show_id, genre_id),
    FOREIGN KEY (show_id) REFERENCES facts_imdb_rating(show_id),
    FOREIGN KEY (genre_id) REFERENCES genre_dim(genre_id)
);
"""
cursor.execute(create_movie_genre)

# Chuẩn bị dữ liệu cho movie_genre
show_genres = []
for _, row in netflix_titles.iterrows():
    movie_id = row['show_id']
    for genre in row['genres']:
        show_genres.append((movie_id, genre_dict[genre]))

# Chèn dữ liệu vào bảng movie_genre
insert_movie_genre = "INSERT INTO show_genre (show_id, genre_id) VALUES (%s, %s);"
cursor.executemany(insert_movie_genre, show_genres)
db.commit()

# Đóng kết nối
db.close()

print("Bảng movie_genre và liên kết đã được thiết lập thành công.")


Bảng movie_genre và liên kết đã được thiết lập thành công.


In [255]:
# create facts_stock_prices table and insert data into it
db = pymysql.connect(
    host="localhost",
    user="root",
    password="",
    database="netflix_datawarehouse"
)
cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS facts_stock_prices;") # delete the table if it already exists

# create table with attributes based on the dimensional model
createTable6 = """CREATE TABLE facts_stock_prices(
                  date_id VARCHAR(50) not null,
                  open FLOAT null,
                  high FLOAT null,
                  low FLOAT null,
                  close FLOAT null,
                  adj_close FLOAT null,
                  volume FLOAT null,
                  PRIMARY KEY (date_id),
                  FOREIGN KEY (date_id) REFERENCES date_dim(date_id)
                  );"""

cursor.execute(createTable6)

insertData6 = "INSERT INTO facts_stock_prices VALUES (%s,%s,%s,%s,%s,%s,%s);" # insert data into table
cursor.executemany(insertData6, factsStockList)
db.commit()

db.close() # close the connection to mysql